In [1]:
from pyecharts.charts import Bar
from pyecharts import options as opts
import pandas as pd

In [6]:
# 读取文件
df = pd.read_excel('./work_all_2.xls')

In [7]:
df

,job_title,company,person_number,salary,path,work,edu,skill
0,数据分析师,北京与车行信息技术有限公司,20-99人,1.2万-1.8万,北京-朝阳,3-5年,本科,数据挖掘数据采集行业分析大数据平台数据监控数据分析
1,数据处理工程师,南京思影科技有限公司,20-99人,1.2万-2万,北京-石景山,不限,本科,MATLABPython机器学习神经网络
2,数据分析,北京中关村科技融资担保有限公司,100-299人,2万-3.5万,北京,5-10年,本科,PythonSQLSASPower BI商业数据分析金融数据分析风控数据分析财务数据分析
3,数据分析师,内蒙古钛驰新迪商务有限公司,20-99人,8千-1.5万,北京-海淀,1年以下,大专,Java商业数据分析运营数据分析财务数据分析风控数据分析游戏数据分析销售数据分析
4,数据分析师（报告写作方向）,北京新媒体(集团)有限公司,100-299人,1万-1.5万,北京-朝阳,1-3年,本科,舆情分析运营数据分析
...,...,...,...,...,...,...,...,...
3670,数据开发工程师（2023届校园招聘）,新意科技,500-999人,150-200/天,福州-鼓楼,无经验,本科,JavaOracle
3671,软件开发中级工程师（数据仓库）,新大陆软件,1000-9999人,8千-1.4万,福州,3-5年,本科,数据仓库
3672,JAVA工程师（大数据开发方向）,福州飞豹电子商务有限公司,20-99人,1万-1.8万,福州,1-3年,本科,MySQLHTMLCSSMemcacheDBSpringBootSpringCloudJav...
3673,大数据开发工程师,江苏正瑞信息科技有限公司,20-99人,1.3万-1.8万,福州-鼓楼,3-5年,大专,数据中台数仓开发JavaHadoopOracleMySQL数据平台HBase


In [8]:
df.fillna(df.person_number.mode()[0])
df.salary = df.salary.str.replace('面议', df.salary.mode()[0])

In [9]:
# 修改单位
df.salary = df.salary.str.replace('180-240/天', '4千-5.2千')
df.salary = df.salary.str.replace('100-150/天', '2.2千-3.3千')
df.salary = df.salary.str.replace('150-200/天', '3.3千-4.4千')
df.salary = df.salary.str.replace('190-200/天', '4千-4.4千')
df.salary = df.salary.str.replace('100-200/天', '2.2千-4.4千')

In [10]:
# 删除非必要
def handle(salary):
    if type(salary) == str:
        if salary[-1] == '万':
            salary = float(salary[:-1]) * 10000
        elif salary[-1] == '千':
            salary = float(salary[:-1]) * 1000
        else:
            salary = float(salary)
    return salary

df[['min_salary', 'max_salary']] = df.salary.str.split('-', 2, expand=True)
df.min_salary = df.min_salary.map(lambda salary: handle(salary))
df.max_salary = df.max_salary.map(lambda salary: handle(salary))
df['avg_salary'] = (df.min_salary + df.max_salary) / 2

E:\Users\panyue\AppData\Local\Temp\ipykernel_17544\18439938.py:12: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['min_salary', 'max_salary']] = df.salary.str.split('-', 2, expand=True)


In [11]:
# 清楚不健康数据
max_num = df.avg_salary.describe([.99])['99%']
df = df[df.avg_salary < max_num]
df.drop(columns=['min_salary', 'salary', 'max_salary'], inplace=True)
df['city'] = df.path.str[:2]
edu_salary = df.groupby(by='edu').avg_salary

E:\Users\panyue\AppData\Local\Temp\ipykernel_17544\1952540481.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['min_salary', 'salary', 'max_salary'], inplace=True)
E:\Users\panyue\AppData\Local\Temp\ipykernel_17544\1952540481.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['city'] = df.path.str[:2]


In [18]:
for i in edu_salary:
    print(i)

('中专/中技', 679      6000.0
1035    21500.0
1544    10500.0
1808     5000.0
1855     5000.0
2246    22500.0
2662    15500.0
2910     7500.0
Name: avg_salary, dtype: float64)
('博士', 1778    25000.0
2017    20000.0
2021    25000.0
2399    30000.0
2404    30000.0
2898    30000.0
Name: avg_salary, dtype: float64)
('大专', 3       11500.0
5       10000.0
8       10000.0
9       10500.0
10      12500.0
         ...   
3635     7500.0
3636     5500.0
3664    10000.0
3668    15000.0
3673    15500.0
Name: avg_salary, Length: 1057, dtype: float64)
('学历不限', 195     11500.0
217     12500.0
266      5500.0
350     12500.0
364     12500.0
421      7500.0
500      9500.0
691      8000.0
712     17000.0
788     18000.0
810      9000.0
1014    12000.0
1237    16500.0
1246     9000.0
1280    11500.0
1447     3500.0
1454    13000.0
1480    16500.0
1663    14500.0
1707    23500.0
1821     6000.0
1836     6500.0
1885    21500.0
1896    12500.0
1945    10500.0
1971    10500.0
2106     5500.0
2460    20000.0
248

In [12]:
df

,job_title,company,person_number,path,work,edu,skill,avg_salary,city
0,数据分析师,北京与车行信息技术有限公司,20-99人,北京-朝阳,3-5年,本科,数据挖掘数据采集行业分析大数据平台数据监控数据分析,15000.0,北京
1,数据处理工程师,南京思影科技有限公司,20-99人,北京-石景山,不限,本科,MATLABPython机器学习神经网络,16000.0,北京
2,数据分析,北京中关村科技融资担保有限公司,100-299人,北京,5-10年,本科,PythonSQLSASPower BI商业数据分析金融数据分析风控数据分析财务数据分析,27500.0,北京
3,数据分析师,内蒙古钛驰新迪商务有限公司,20-99人,北京-海淀,1年以下,大专,Java商业数据分析运营数据分析财务数据分析风控数据分析游戏数据分析销售数据分析,11500.0,北京
4,数据分析师（报告写作方向）,北京新媒体(集团)有限公司,100-299人,北京-朝阳,1-3年,本科,舆情分析运营数据分析,12500.0,北京
...,...,...,...,...,...,...,...,...,...
3670,数据开发工程师（2023届校园招聘）,新意科技,500-999人,福州-鼓楼,无经验,本科,JavaOracle,3850.0,福州
3671,软件开发中级工程师（数据仓库）,新大陆软件,1000-9999人,福州,3-5年,本科,数据仓库,11000.0,福州
3672,JAVA工程师（大数据开发方向）,福州飞豹电子商务有限公司,20-99人,福州,1-3年,本科,MySQLHTMLCSSMemcacheDBSpringBootSpringCloudJav...,14000.0,福州
3673,大数据开发工程师,江苏正瑞信息科技有限公司,20-99人,福州-鼓楼,3-5年,大专,数据中台数仓开发JavaHadoopOracleMySQL数据平台HBase,15500.0,福州


In [19]:
edu_salary_avg = edu_salary.mean()
# 取出学历和对应平均薪资
avg_salary_edu = edu_salary_avg.index.tolist()
avg_value_edu = edu_salary_avg.values.tolist()

In [20]:
avg_salary_edu

['中专/中技', '博士', '大专', '学历不限', '本科', '硕士', '高中']

In [21]:
avg_value_edu

[11687.5,
 26666.666666666668,
 9761.684011352885,
 10744.186046511628,
 13345.123482628715,
 14277.34375,
 8625.0]

In [25]:
A = (
    Bar()
    .add_xaxis(avg_salary_edu)
    .add_yaxis("平均薪资",avg_value_edu)
    .set_global_opts(
        title_opts=opts.TitleOpts(title='学历水平对平均薪资的影响'),
        yaxis_opts=opts.AxisOpts(name='平均薪资（单位：元）'),
        xaxis_opts=opts.AxisOpts(name='学历水平')
    )
    .render_notebook()
#    .render("学历水平对平均薪资的影响.html")
)

In [28]:
A